In [3]:
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
from shapely import geometry
import pandas as pd

# split precinct into 10 files

In [4]:
gdf = gpd.read_file("all_states_precincts_cname_neighbor_demo.json")
gdf = gdf[gdf["State"] == "va"]
gdf.head(1)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,Neighbors,Total Population,White,Black or African American,American Indian,Asian,Other Population,geometry
0,Accomack,0001,486,637,16,8,4,2328,va,va-York-2446,"va-York-2446,va-York-2446; va-York-2446,va-Yor...",3526,2336,906,12,20,246,"POLYGON ((-75.66233 37.77053, -75.66244 37.770..."


In [5]:
counties = sorted(list(gdf["County"].unique()))
print(len(counties))

113


In [6]:
# test, should be 28
len(gdf[gdf["County"].isin(counties[9*11:])]["County"].unique())

14

In [7]:
# split to 4 files with 28 counties each file (last file 29)
for i in range(10):
    if i == 9:
        file_name = "VA" + str(i) + "Precinct.json"
        gdf[gdf["County"].isin(counties[i*11 :])].to_file(file_name, driver="GeoJSON")
    else:
        file_name = "VA" + str(i) + "Precinct.json"
        gdf[gdf["County"].isin(counties[i*11 : (i+1)*11])].to_file(file_name, driver="GeoJSON")

In [2]:
113//10

11

# Start parsing precinct data 

In [2]:
Polygon = geometry.polygon.Polygon
MultiPolygon = geometry.multipolygon.MultiPolygon

In [3]:
def parse(gdf, arg_i, arg_shape_id, arg_polygon_id):
    columns = ["geovertex_id", "shape_id", "polygon_id", "CName", "x_pos", "y_pos"]
    df = pd.DataFrame(columns=columns)

    counter = 0
    i = arg_i
    shape_id = arg_shape_id
    polygon_id = arg_polygon_id

    for index, row in gdf.iterrows():
        # for visualization only
        if counter%15==0:
            print(" {:.2f}% ".format(counter/len(gdf)*100), end="")
        else:
            print("#", end="")

        cur_shape = row["geometry"]
        cur_type = type(cur_shape)

        row = row.drop(["geometry"])
        cur_row = row
        cur_row["shape_id"] = shape_id
        shape_id += 1

        if cur_type == MultiPolygon:
            for polygon in cur_shape:
                cur_row["polygon_id"] = polygon_id
                polygon_id += 1
                for coords in polygon.exterior.coords:
                    cur_row["geovertex_id"] = i
                    i += 1
                    cur_row["x_pos"] = coords[0]
                    cur_row["y_pos"] = coords[1]
                    df = df.append(cur_row, ignore_index=True)
                    cur_row = row
        elif cur_type == Polygon:
            cur_row["polygon_id"] = polygon_id
            polygon_id += 1
            for coords in cur_shape.exterior.coords:
                cur_row["geovertex_id"] = i
                i += 1
                cur_row["x_pos"] = coords[0]
                cur_row["y_pos"] = coords[1]
                df = df.append(cur_row, ignore_index=True)
                cur_row = row
        counter += 1
        
    return df

In [ ]:
next_i, next_shape, next_poly = 1495449, 959, 1940 #geo:2078126, shape:1438, poly:2427 VA6Precinct.json
for i in range(4, 10):
    file_name = "VA" + str(i) + "Precinct.json"
    
    print("\n\nStart", file_name)
    gdf = gpd.read_file(file_name)
    
    # drop some columns
    dropping = gdf.columns.to_list()
    dropping.remove("geometry")
    dropping.remove("CName")
    gdf.drop(dropping, inplace=True, axis=1)
    
    # parsing
    gdf_Parsed = parse(gdf, arg_i=next_i, arg_shape_id=next_shape, arg_polygon_id=next_poly)
    
    # saving the result 
    file_name = file_name.split(".")[0]
    gdf_Parsed.to_csv(file_name+"_Parsed.csv")

    next_i = gdf_Parsed.iloc[-1]["geovertex_id"] + 1
    next_shape = gdf_Parsed.iloc[-1]["shape_id"] + 1
    next_poly = gdf_Parsed.iloc[-1]["polygon_id"] + 1
    print("\nLast row id: geo:{}, shape:{}, poly:{}".format(gdf_Parsed.iloc[-1]["geovertex_id"],
                                                         gdf_Parsed.iloc[-1]["shape_id"],
                                                         gdf_Parsed.iloc[-1]["polygon_id"]))
    print("Next id: geo:{}, shape:{}, poly:{}".format(next_i, next_shape, next_poly))



Start VA4Precinct.json
 0.00% ############## 5.47% ############## 10.95% ############## 16.42% ############## 21.90% ############## 27.37% ############## 32.85% ############## 38.32% ############## 43.80% ############## 49.27% ############## 54.74% ############## 60.22% ############## 65.69% ############## 71.17% ############## 76.64% ############## 82.12% ############## 87.59% ############## 93.07% ############## 98.54% ###
Last row id: geo:1780454, shape:1232, poly:2220
Next id: geo:1780455, shape:1233, poly:2221


Start VA5Precinct.json
 0.00% ############## 7.32% ############## 14.63% ############## 21.95% ############## 29.27% ############## 36.59% ############## 43.90% ############## 51.22% ############## 58.54% ############## 65.85% ############## 73.17% ############## 80.49% ############## 87.80% ############## 95.12% #########
Last row id: geo:2078125, shape:1437, poly:2426
Next id: geo:2078126, shape:1438, poly:2427


Start VA6Precinct.json
 0.00% ########